In [1]:
import os
import sys
import copy
import time
import random
import pyspark
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pyspark.sql.functions as F
from pyspark.sql import SparkSession
from pyspark.sql.types import DoubleType, FloatType
from pyspark.sql.functions import col

# for creating pipelines and model
from pyspark.ml import Pipeline, PipelineModel
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler, VectorIndexer
from pyspark.ml import Pipeline, PipelineModel
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# sklearn
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

In [2]:
import os
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql import SparkSession
from pyspark.sql.types import FloatType, IntegerType, StructField, StructType
from time import time
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.functions import col
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars xgboost4j_3.0-1.3.0-0.1.0.jar,xgboost4j-spark_3.0-1.3.0-0.1.0.jar,cudf-0.18.1-cuda11.jar,rapids-4-spark_2.12-0.4.1.jar pyspark-shell'

def init_spark():
    spark = SparkSession \
        .builder \
        .getOrCreate()
    return spark

spark = init_spark()

# Data Split

In [3]:
filename = "..\data\machines.csv"
machines = spark.read.csv(filename, sep=',', header=True)

print(machines.count())
machines.show()

100
+---------+------+---+
|machineID| model|age|
+---------+------+---+
|        1|model3| 18|
|        2|model4|  7|
|        3|model3|  8|
|        4|model3|  7|
|        5|model3|  2|
|        6|model3|  7|
|        7|model3| 20|
|        8|model3| 16|
|        9|model4|  7|
|       10|model3| 10|
|       11|model2|  6|
|       12|model3|  9|
|       13|model1| 15|
|       14|model3|  1|
|       15|model3| 14|
|       16|model1|  3|
|       17|model1| 14|
|       18|model3| 15|
|       19|model3| 17|
|       20|model2| 16|
+---------+------+---+
only showing top 20 rows



In [3]:
feat_data = spark.read.parquet('../data/labeled_features.parquet')
feat_data = feat_data.withColumn("age", feat_data.age.cast(DoubleType()))

print(feat_data.count())
# highly imbalanced data
print(feat_data.groupby('label_e').count().show())
feat_data.limit(10).toPandas().head(10)

73142
+-------+-----+
|label_e|count|
+-------+-----+
|    0.0|67470|
|    1.0| 1337|
|    4.0| 1473|
|    3.0| 1000|
|    2.0| 1862|
+-------+-----+

None


,dt_truncated,volt_rollingmean_12,rotate_rollingmean_12,pressure_rollingmean_12,vibration_rollingmean_12,volt_rollingmean_24,rotate_rollingmean_24,pressure_rollingmean_24,vibration_rollingmean_24,volt_rollingmean_36,...,error5sum_rollingmean_24,comp1sum,comp2sum,comp3sum,comp4sum,model,age,model_encoded,failure,label_e
0,2016-01-01 07:00:00,172.083928,453.576897,101.303110,40.627410,169.230878,451.007306,100.487259,40.839262,167.339602,...,0.0,579.0,534.0,474.0,459.0,model4,3.0,"(0.0, 1.0, 0.0)",0.0,0.0
1,2015-12-31 19:00:00,168.173348,453.181951,99.527531,40.981132,165.787189,449.842118,100.598808,41.791947,166.190766,...,0.0,578.0,533.0,473.0,458.0,model4,3.0,"(0.0, 1.0, 0.0)",0.0,0.0
2,2015-12-31 07:00:00,163.401030,446.502286,101.670084,42.602762,165.199475,445.038344,101.074817,41.722713,168.995817,...,0.0,578.0,533.0,473.0,458.0,model4,3.0,"(0.0, 1.0, 0.0)",0.0,0.0
3,2015-12-30 19:00:00,166.997919,443.574402,100.479550,40.842664,171.793211,450.456864,100.955598,40.418503,172.419415,...,0.0,577.0,532.0,472.0,457.0,model4,3.0,"(0.0, 1.0, 0.0)",0.0,0.0
4,2015-12-30 07:00:00,176.588502,457.339327,101.431647,39.994342,175.130162,466.483595,99.468624,40.920312,174.914358,...,0.0,577.0,532.0,472.0,457.0,model4,3.0,"(0.0, 1.0, 0.0)",0.0,0.0
5,2015-12-29 19:00:00,173.671822,475.627863,97.505602,41.846282,174.077285,464.681249,99.852795,41.520096,173.561514,...,0.0,576.0,531.0,471.0,456.0,model4,3.0,"(0.0, 1.0, 0.0)",0.0,0.0
6,2015-12-29 07:00:00,174.482749,453.734636,102.199989,41.193909,173.506360,450.268077,101.890206,39.760938,173.318924,...,0.0,576.0,531.0,471.0,456.0,model4,3.0,"(0.0, 1.0, 0.0)",0.0,0.0
7,2015-12-28 19:00:00,172.529971,446.801518,101.580424,38.327966,172.737011,443.114257,99.159233,38.946824,171.416655,...,0.0,575.0,530.0,470.0,455.0,model4,3.0,"(0.0, 1.0, 0.0)",0.0,0.0
8,2015-12-28 07:00:00,172.944052,439.426996,96.738042,39.565681,170.859997,444.430674,99.034004,39.874349,172.384220,...,0.0,575.0,530.0,470.0,455.0,model4,3.0,"(0.0, 1.0, 0.0)",0.0,0.0
9,2015-12-27 19:00:00,168.775941,449.434352,101.329965,40.183016,172.104304,444.571586,98.413556,39.550401,170.807337,...,0.0,574.0,529.0,469.0,454.0,model4,3.0,"(0.0, 1.0, 0.0)",0.0,0.0


In [4]:
label_var = ['label_e']
key_cols =['machineID','dt_truncated']
input_features = feat_data.columns
remove_cols = label_var + key_cols + ['failure','model_encoded','model' ]

# Remove the extra names if that are in the input_features list
input_features = [x for x in input_features if x not in set(remove_cols)]
# Use cols
input_features

['volt_rollingmean_12',
 'rotate_rollingmean_12',
 'pressure_rollingmean_12',
 'vibration_rollingmean_12',
 'volt_rollingmean_24',
 'rotate_rollingmean_24',
 'pressure_rollingmean_24',
 'vibration_rollingmean_24',
 'volt_rollingmean_36',
 'vibration_rollingmean_36',
 'rotate_rollingmean_36',
 'pressure_rollingmean_36',
 'volt_rollingstd_12',
 'rotate_rollingstd_12',
 'pressure_rollingstd_12',
 'vibration_rollingstd_12',
 'volt_rollingstd_24',
 'rotate_rollingstd_24',
 'pressure_rollingstd_24',
 'vibration_rollingstd_24',
 'volt_rollingstd_36',
 'rotate_rollingstd_36',
 'pressure_rollingstd_36',
 'vibration_rollingstd_36',
 'error1sum_rollingmean_24',
 'error2sum_rollingmean_24',
 'error3sum_rollingmean_24',
 'error4sum_rollingmean_24',
 'error5sum_rollingmean_24',
 'comp1sum',
 'comp2sum',
 'comp3sum',
 'comp4sum',
 'age']

In [5]:
# assemble features
va = VectorAssembler(inputCols=(input_features), outputCol='features')
feat_data = va.transform(feat_data).select('machineID','dt_truncated','label_e','features')

# set maxCategories so features with > 10 distinct values are treated as continuous.
featureIndexer = VectorIndexer(inputCol="features", 
                               outputCol="indexedFeatures", 
                               maxCategories=10).fit(feat_data)

# fit on whole dataset to include all labels in index
labelIndexer = StringIndexer(inputCol="label_e", outputCol="indexedLabel").fit(feat_data)

# split the data into train/test based on date
split_date = "2015-10-30"
training = feat_data.filter(feat_data.dt_truncated < split_date)
testing = feat_data.filter(feat_data.dt_truncated >= split_date)

print(training.count())
print(testing.count())

60434
12708


In [ ]:
# Down sample majority class, do we really need this?
# SampleBy returns a stratified sample without replacement based on the fraction given on each stratum
train_downsampled = training.sampleBy('label', fractions={0.0: 0.135, 1.0: 1.0}, seed=123).cache()
train_downsampled.groupby('label').count().show()

testing.groupby('label').count().show()

In [ ]:
# Cache results
# cache datasets in memory
train_downsampled.cache()
testing.cache()

# check the number of devices in training and testing data
print(train_downsampled.select('deviceid').distinct().count())
print(testing.select('deviceid').distinct().count())

# GBT Gradient-Boosted Tree

In [26]:
# GBTClassifier currently only supports binary classification.
training.dtypes

[('machineID', 'string'),
 ('dt_truncated', 'timestamp'),
 ('label_e', 'double'),
 ('features', 'vector')]

In [ ]:
%%time

# Train a GBT model.
gbt = GBTClassifier(labelCol='label_e', featuresCol='features', maxDepth=10, minInstancesPerNode=5, maxIter=50)

# Chain indexers and GBT in a Pipeline
pipeline_gbt = Pipeline(stages=[labelIndexer, featureIndexer, gbt])

# Train model. This also runs the indexers.
model_gbt = pipeline_gbt.fit(training)

# save model
datestamp = unicode(datetime.datetime.now()).replace(' ','').replace(':','_');
gbt_fileName = '../checkpoints/GradientBoostedTree_' + datestamp;
gbtDirfilename = modelDir + gbt_fileName;
model_gbt.save(gbtDirfilename)

# Make predictions.
predictions_gbt = model_gbt.transform(testing)

# GBT eXtreme Gradient Boosting

In [5]:
#!wget https://repo1.maven.org/maven2/com/nvidia/xgboost4j_3.0/1.0.0-0.1.0/xgboost4j_3.0-1.0.0-0.1.0.jar
#!wget https://repo1.maven.org/maven2/com/nvidia/xgboost4j-spark_3.0/1.0.0-0.1.0/xgboost4j-spark_3.0-1.0.0-0.1.0.jar
#!wget http://insecure.repo1.maven.org/maven2/com/nvidia/rapids-4-spark_2.12/0.1.0/rapids-4-spark_2.12-0.1.0.jar
#!wget https://repo1.maven.org/maven2/ai/rapids/cudf/0.18/cudf-0.18-cuda10-1.jar --no-check-certificate

#!wget https://raw.githubusercontent.com/srivatsan88/YoutubeLI/master/dataset/covtype_train.parquet
#!wget https://raw.githubusercontent.com/srivatsan88/YoutubeLI/master/dataset/covtype_test.parquet


--2021-03-26 23:59:35--  https://repo1.maven.org/maven2/ai/rapids/cudf/0.18/cudf-0.18-cuda10-1.jar
Resolving repo1.maven.org (repo1.maven.org)... 151.101.124.209
Connecting to repo1.maven.org (repo1.maven.org)|151.101.124.209|:443... connected.
  Unable to locally verify the issuer's authority.
HTTP request sent, awaiting response... 200 OK
Length: 314947979 (300M) [application/java-archive]
Saving to: 'cudf-0.18-cuda10-1.jar'

     0K .......... .......... .......... .......... ..........  0%  279K 18m22s
    50K .......... .......... .......... .......... ..........  0% 1.57M 10m47s
   100K .......... .......... .......... .......... ..........  0%  470K 10m49s
   150K .......... .......... .......... .......... ..........  0%  430K 11m6s
   200K .......... .......... .......... .......... ..........  0%  263K 12m46s
   250K .......... .......... .......... .......... ..........  0% 93.3K 19m47s
   300K .......... .......... .......... .......... ..........  0% 79.4K 26m10s
   350K .

  5900K .......... .......... .......... .......... ..........  1% 1.18M 37m13s
  5950K .......... .......... .......... .......... ..........  1% 1.34M 36m56s
  6000K .......... .......... .......... .......... ..........  1% 1.48M 36m39s
  6050K .......... .......... .......... .......... ..........  1% 1.14M 36m22s
  6100K .......... .......... .......... .......... ..........  1% 1.65M 36m6s
  6150K .......... .......... .......... .......... ..........  2% 1.54M 35m49s
  6200K .......... .......... .......... .......... ..........  2% 1.69M 35m33s
  6250K .......... .......... .......... .......... ..........  2% 1.79M 35m17s
  6300K .......... .......... .......... .......... ..........  2% 1.90M 35m1s
  6350K .......... .......... .......... .......... ..........  2% 2.03M 34m46s
  6400K .......... .......... .......... .......... ..........  2% 1.87M 34m31s
  6450K .......... .......... .......... .......... ..........  2%  707K 34m18s
  6500K .......... .......... .......... .

 39000K .......... .......... .......... .......... .......... 12% 3.12M 11m13s
 39050K .......... .......... .......... .......... .......... 12% 2.87M 11m12s
 39100K .......... .......... .......... .......... .......... 12% 2.85M 11m11s
 39150K .......... .......... .......... .......... .......... 12% 3.80M 11m10s
 39200K .......... .......... .......... .......... .......... 12% 3.13M 11m9s
 39250K .......... .......... .......... .......... .......... 12% 2.33M 11m8s
 39300K .......... .......... .......... .......... .......... 12% 3.17M 11m7s
 39350K .......... .......... .......... .......... .......... 12% 2.98M 11m7s
 39400K .......... .......... .......... .......... .......... 12% 3.32M 11m6s
 39450K .......... .......... .......... .......... .......... 12% 2.98M 11m5s
 39500K .......... .......... .......... .......... .......... 12% 3.16M 11m4s
 39550K .......... .......... .......... .......... .......... 12% 3.17M 11m3s
 39600K .......... .......... .......... .......

149500K .......... .......... .......... .......... .......... 48% 3.11M 2m22s
149550K .......... .......... .......... .......... .......... 48% 2.92M 2m22s
149600K .......... .......... .......... .......... .......... 48% 3.46M 2m21s
149650K .......... .......... .......... .......... .......... 48% 2.26M 2m21s
149700K .......... .......... .......... .......... .......... 48% 3.38M 2m21s
149750K .......... .......... .......... .......... .......... 48% 3.10M 2m21s
149800K .......... .......... .......... .......... .......... 48% 3.03M 2m21s
149850K .......... .......... .......... .......... .......... 48% 3.26M 2m21s
149900K .......... .......... .......... .......... .......... 48% 2.97M 2m21s
149950K .......... .......... .......... .......... .......... 48% 3.16M 2m21s
150000K .......... .......... .......... .......... .......... 48% 2.94M 2m21s
150050K .......... .......... .......... .......... .......... 48% 2.49M 2m21s
150100K .......... .......... .......... .......... 

174450K .......... .......... .......... .......... .......... 56% 2.33M 1m48s
174500K .......... .......... .......... .......... .......... 56% 3.15M 1m48s
174550K .......... .......... .......... .......... .......... 56% 3.17M 1m48s
174600K .......... .......... .......... .......... .......... 56% 3.30M 1m48s
174650K .......... .......... .......... .......... .......... 56% 3.10M 1m48s
174700K .......... .......... .......... .......... .......... 56% 3.07M 1m48s
174750K .......... .......... .......... .......... .......... 56% 3.07M 1m48s
174800K .......... .......... .......... .......... .......... 56% 3.09M 1m48s
174850K .......... .......... .......... .......... .......... 56% 2.45M 1m48s
174900K .......... .......... .......... .......... .......... 56% 3.03M 1m48s
174950K .......... .......... .......... .......... .......... 56% 3.18M 1m48s
175000K .......... .......... .......... .......... .......... 56% 3.00M 1m48s
175050K .......... .......... .......... .......... 

304600K .......... .......... .......... .......... .......... 99% 3.09M 2s
304650K .......... .......... .......... .......... .......... 99% 3.10M 2s
304700K .......... .......... .......... .......... .......... 99% 3.27M 2s
304750K .......... .......... .......... .......... .......... 99% 3.01M 2s
304800K .......... .......... .......... .......... .......... 99% 2.93M 2s
304850K .......... .......... .......... .......... .......... 99% 2.53M 2s
304900K .......... .......... .......... .......... .......... 99% 3.15M 2s
304950K .......... .......... .......... .......... .......... 99% 3.14M 2s
305000K .......... .......... .......... .......... .......... 99% 3.07M 2s
305050K .......... .......... .......... .......... .......... 99% 3.03M 1s
305100K .......... .......... .......... .......... .......... 99% 3.21M 1s
305150K .......... .......... .......... .......... .......... 99% 2.78M 1s
305200K .......... .......... .......... .......... .......... 99% 3.61M 1s
305250K ....

In [3]:
spark.sparkContext.addPyFile("xgboost4j-spark_3.0-1.0.0-0.1.0.jar")
spark.sparkContext.addPyFile("rapids-4-spark_2.12-0.1.0.jar")

from ml.dmlc.xgboost4j.scala.spark import XGBoostClassificationModel, XGBoostClassifier
#training.dtypes

In [7]:
# setting parameters. you can find these parameters in the link above.
params = {'eta': 0.1, 'gamma': 0.1, 'missing': 0.0,
          'treeMethod': 'gpu_hist', 'maxDepth': 3, 
          'growPolicy': 'depthwise', 'lambda_': 1.0,
          'subsample': 1.0, 'numRound': 1000,
          'numWorkers': 1, 'verbosity': 1}

features = [feat for feat in feat_data.columns if feat != 'label_e']

# create XGBoost model object
# xgboost = XGBoostClassifier()\
#             .setLabelCol('label_e')\
#             .setFeaturesCols(features)

xgboost = XGBoostClassifier(
    featuresCol="features", 
    labelCol="label_e", 
)
xgboost.setMissing(0.0)

# # getting the feature names
# target = 'label_e'
# features = [feat for feat in feat_data.columns if feat != target]

# # setting parameters. you can find these parameters in the link above.
# params = {'eta': 0.1, 'gamma': 0.1, 'missing': 0.0,
#           'treeMethod': 'gpu_hist', 'maxDepth': 3, 
#           'growPolicy': 'depthwise', 'lambda_': 1.0,
#           'subsample': 1.0, 'numRound': 1000,
#           'numWorkers': 1, 'verbosity': 1}

# # create XGBoost model object
# xgboost = XGBoostClassifier(**params)\
#             .setLabelCol(target)\
#             .setFeaturesCols(features)

# xgboost.setMaxDepth(2)

XGBoostClassifier_6d72fffbfc9d

In [ ]:
start_time = time.time()

model = xgboost.fit(training)

print("Training Time: %s seconds" % (str(time.time() - start_time)))


In [6]:
spark.sparkContext.addPyFile("xgboost4j-spark_3.0-1.3.0-0.1.0.jar")
spark.sparkContext.addPyFile("rapids-4-spark_2.12-0.4.1.jar")

from ml.dmlc.xgboost4j.scala.spark import XGBoostClassificationModel, XGBoostClassifier


# loading data
reader = spark.read
train_data = reader.parquet("covtype_train.parquet")
test_data = reader.parquet("covtype_test.parquet")

# getting the feature names
target = 'target'
features = [feat for feat in train_data.schema.names if feat != target]

# CPU
def vectorize(data_frame):
    to_floats = [ col(x.name).cast(FloatType()) for x in data_frame.schema ]
    return (VectorAssembler()
        .setInputCols(features)
        .setOutputCol('features')
        .transform(data_frame.select(to_floats))
        .select(col('features'), col('target')))

train_data = vectorize(train_data)

# setting parameters. you can find these parameters in the link above.
params = { 
    'eta': 0.1, 'gamma': 0.1, 'missing': 0.0,
    'treeMethod': 'hist', 'maxDepth': 10, 
    'maxLeaves': 256, 'growPolicy': 'depthwise',
    'minChildWeight': 30.0, 'lambda_': 1.0,
    'scalePosWeight': 2.0, 'subsample': 1.0,
    'nthread': 1, 'numWorkers': 1,
}

# create XGBoost model object
xgboost = XGBoostClassifier(**params)\
            .setLabelCol('target')\
            .setFeaturesCol('features')
train_data.limit(10).toPandas().head(10)

,features,target
0,"(3381.0, 0.0, 18.0, 497.0, 187.0, 4201.0, 190....",6.0
1,"(3039.0, 291.0, 12.0, 361.0, 87.0, 618.0, 187....",1.0
2,"(3318.0, 353.0, 21.0, 309.0, 84.0, 4548.0, 179...",0.0
3,"(2823.0, 67.0, 13.0, 170.0, 54.0, 2514.0, 233....",1.0
4,"(3211.0, 16.0, 19.0, 350.0, 36.0, 3254.0, 200....",1.0
5,"(2797.0, 129.0, 9.0, 0.0, 0.0, 2400.0, 235.0, ...",1.0
6,"(3385.0, 282.0, 8.0, 899.0, 179.0, 1092.0, 198...",0.0
7,"(3065.0, 0.0, 4.0, 242.0, 10.0, 3298.0, 214.0,...",0.0
8,"(3209.0, 157.0, 7.0, 309.0, 46.0, 5057.0, 228....",0.0
9,"(3004.0, 151.0, 18.0, 90.0, 3.0, 3458.0, 240.0...",1.0


In [4]:
# train the model
model = xgboost.fit(train_data)


Py4JJavaError: An error occurred while calling o224.fit.
: ml.dmlc.xgboost4j.java.XGBoostError: XGBoostModel training failed
	at ml.dmlc.xgboost4j.scala.spark.XGBoost$.postTrackerReturnProcessing(XGBoost.scala:749)
	at ml.dmlc.xgboost4j.scala.spark.XGBoost$.trainDistributed(XGBoost.scala:623)
	at ml.dmlc.xgboost4j.scala.spark.XGBoostClassifier.train(XGBoostClassifier.scala:218)
	at ml.dmlc.xgboost4j.scala.spark.XGBoostClassifier.train(XGBoostClassifier.scala:41)
	at org.apache.spark.ml.Predictor.fit(Predictor.scala:150)
	at ml.dmlc.xgboost4j.scala.spark.XGBoostClassifier.fit(XGBoostClassifier.scala:171)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at java.base/jdk.internal.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.base/java.lang.reflect.Method.invoke(Method.java:566)
	at py4j.reflection.Me